In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from h5py import File
import pandas as pd
from scipy.signal import savgol_filter
from scipy import signal

In [ ]:
run=86
raw = File("/reg/data/ana15/tmo/tmolv2918/scratch/preproc/v5/run%i_v5.h5" % run,'r')

In [ ]:
# get some data
order = raw['timestamp'][:].argsort()  # get and sort timestamps
jet   = raw['evrs'][:,70][order].astype(bool)  # jet on or off; sort array by timestamps
bykik = raw['evrs'][:,161].astype(bool)[order]  # no x-rays?
goose = raw['evrs'][:,68].astype(bool)[order]  # UV off ?

gmd     = raw['gmd_energy'][:][order]  # gmd after attenuator
xgmd    = raw['xgmd_energy'][:][order]  # gmd after attenuator
photes  = raw['ebeam_ebeamPhotonEnergy'][:][order]  # nominal photon energies
l3s     = raw['ebeam_ebeamL3Energy'][:][order]  # l3 beam energy
vls     = raw["vls"][:][order]  # spectra from vls spectrometer
itof_wf = raw["iToF_wf"][:][order]  # itof spectra

jet_delay   = raw['epics_lamp_jet_delay'][:][order]  # what is that??
laser_delay = raw['epics_las_fs14_target_time'][:][order]  # laser delay :O

# time of flight taken from elio
tflight = np.arange(itof_wf.shape[1]) * 10 / 60 / 100
gmdFilt = gmd>-0.5

l3_filter_low = l3s>5670
l3_filter_up = l3s < 5700

#tof to mass:
c, t0 = 1.315, 0.198 
m_qs = ((tflight - t0) / c)**2

# get rid shots with no gas or x-rays
itof        = itof_wf[jet&~bykik&~goose&gmdFilt]
goose_itof  = itof_wf[jet&~bykik&goose&gmdFilt]
delays = laser_delay[jet&~bykik&~goose&gmdFilt]
xgmdF  = xgmd[jet&~bykik&~goose&gmdFilt]
vls    = pd.DataFrame(np.squeeze(vls[jet&~bykik&~goose&gmdFilt]))
l3s    = pd.DataFrame(np.squeeze(l3s[jet&~bykik&~goose&gmdFilt]))
photes = photes[jet&~bykik&~goose&gmdFilt]

itof_nokick_gas=itof_wf[~bykik&l3_filter_up&l3_filter_low&jet]#this selects also on jet
itof_nokick_nogas=itof_wf[~bykik&l3_filter_up&l3_filter_low&~jet]#this
newgmd_gas=xgmd[~bykik&l3_filter_up&l3_filter_low&jet]#this
newgmd_gas_nogas=xgmd[~bykik&l3_filter_up&l3_filter_low&jet]#this

iev    = pd.DataFrame(itof, columns=m_qs) 